In [1]:
import os
import json
import time
import uuid
from typing import List, Dict, Any
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm

In [2]:
from openai import OpenAI

In [3]:
load_dotenv()
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')
if not DEEPSEEK_API_KEY:
    raise EnvironmentError('Set DEEPSEEK_API_KEY in your environment')

In [4]:
client = OpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1" # Note: Use /v1 for openai client v1+
)

In [5]:
MEMORY_FILE = 'memory_store.json'
CHAT_MODEL = 'deepseek-chat'
TOP_K = 3

In [6]:
def ensure_memory_file():
    if not os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'w') as f:
            json.dump([], f)

In [7]:
def load_memory() -> List[Dict[str, Any]]:
    ensure_memory_file()
    with open(MEMORY_FILE, 'r') as f:
        return json.load(f)

In [8]:
def save_memory(mem: List[Dict[str, Any]]):
    with open(MEMORY_FILE, 'w') as f:
        json.dump(mem, f, indent=2)

In [9]:
def embed_text(text: str) -> List[float]:
    np.random.seed(abs(hash(text)) % (2**32))
    return np.random.rand(512).tolist()


In [10]:
def cosine_similarity(a: List[float], b: List[float]) -> float:
    a = np.array(a, dtype=np.float32)
    b = np.array(b, dtype=np.float32)
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [11]:
def retrieve_similar(query: str, top_k: int = TOP_K) -> List[Dict[str, Any]]:
    mem = load_memory()
    if not mem:
        return []
    q_emb = embed_text(query)
    scored = []
    for entry in mem:
        score = cosine_similarity(q_emb, entry['embedding'])
        scored.append((score, entry))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [e for s,e in scored[:top_k]]

In [12]:
def add_memory(text: str, embedding: List[float]):
    mem = load_memory()
    entry = {
        'id': str(uuid.uuid4()),
        'text': text,
        'embedding': embedding,
        'ts': time.time()
    }
    mem.append(entry)
    save_memory(mem)

In [13]:
def build_prompt(user_input: str, retrieved: List[Dict[str, Any]]) -> str:
    prompt = "You are a helpful assistant. Use the relevant memories below when answering.\n\n"
    if retrieved:
        prompt += "Relevant memories:\n"
        for i, r in enumerate(retrieved):
            prompt += f"[{i+1}] {r['text']}\n"
        prompt += "\n"
    prompt += f"User: {user_input}\nAssistant:"
    return prompt

In [14]:
def call_llm(prompt: str) -> str:
    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[{'role':'system','content':'You are a concise helpful agent.'},
                  {'role':'user','content':prompt}],
        max_tokens=300,
        temperature=0.2
    )
    reply = resp.choices[0].message.content.strip()
    return reply

In [15]:
def interactive():
    print('=== Minimal Memory-Augmented Chat Agent Demo (DeepSeek) ===')
    print('(type "exit" to quit, "showmem" to view memory)')
    ensure_memory_file()
    while True:
        user_input = input('\nYou: ').strip()
        if user_input.lower() in ('exit','quit'):
            break
        if user_input.lower() == 'showmem':
            mem = load_memory()
            if not mem:
                print('Memory is empty')
            else:
                for i,e in enumerate(mem):
                    print(f"{i+1}. {e['text']} (ts={int(e['ts'])})")
            continue

        retrieved = retrieve_similar(user_input)
        prompt = build_prompt(user_input, retrieved)
        
        print('\nAssistant: ...thinking...')
        reply = call_llm(prompt)
        print('\rAssistant:', reply, " " * 10) 

        pair_text = f"User: {user_input} | Assistant: {reply}"
        try:
            emb = embed_text(pair_text)
            add_memory(pair_text, emb)
        except Exception as e:
            print('Warning: failed to embed/save memory:', e)

In [16]:
if __name__ == '__main__':
    interactive()

=== Minimal Memory-Augmented Chat Agent Demo (DeepSeek) ===
(type "exit" to quit, "showmem" to view memory)

Assistant: ...thinking...
Assistant: Hello! How can I assist you today?           

Assistant: ...thinking...
Assistant: Based on our previous conversation, I see you're feeling bored. Here are some engaging activities you could try:

• Learn a new skill online (like a language or instrument)
• Start a creative project (drawing, writing, cooking)
• Go for a walk or exercise
• Listen to music or a podcast
• Connect with friends or family
• Play a game or puzzle
• Read a book or watch a documentary

What type of activity interests you most? I can provide more specific suggestions!           

Assistant: ...thinking...
Assistant: Based on our previous conversation where I suggested various activities to combat boredom, I notice you haven't specified your preferences yet. To give you more tailored suggestions, could you tell me what type of activity might interest you? For example:


KeyboardInterrupt: 